# Codebase - Custom Transformer Training

In [1]:
import torch
from codebase.model import Model
from codebase.data import Dataset
from codebase.train import train, train_exhaustively
from codebase.inference import generate, tokens_to_segs
from codebase.utils import load_dataset, visualize_model, visualize_teacher_forcing, plot_loss, save_pkl
from codebase.preprocessing import create_dataset
from codebase.evaluate import evaluate
from codebase.miscellaneous.explore_curve_families import plot_track_segments

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [ ]:
'''
dataset = create_dataset(
    split="test",
    dataset_path="maestro-v3.0.0", 
    seg_fit_tightness=0.12,
    nocturnes=False,
    track_idx=None,
    num_workers=-1,
    max_chunk_size_mb=400,
    output_file="test_set"
    )
'''

Loading paths...
Found 19 tracks
Using 8 parallel workers
[1/19] MIDI-Unprocessed_R1_D1-9-12_mid--AUDIO-from_mp3_12_R1_2015_wav--3.midi | 10 augmented versions | Avg: 10.5s/track | ETA: 3.1min
[2/19] MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_04_R1_2015_wav--3.midi | 10 augmented versions | Avg: 5.4s/track | ETA: 1.5min
[3/19] MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_01_R1_2015_wav--2.midi | 10 augmented versions | Avg: 4.0s/track | ETA: 1.1min
[4/19] MIDI-Unprocessed_R1_D2-21-22_mid--AUDIO-from_mp3_22_R1_2015_wav--4.midi | 10 augmented versions | Avg: 3.1s/track | ETA: 0.8min
[5/19] MIDI-Unprocessed_22_R2_2011_MID--AUDIO_R2-D5_10_Track10_wav.midi | 10 augmented versions | Avg: 2.6s/track | ETA: 0.6min
[6/19] MIDI-Unprocessed_072_PIANO072_MID--AUDIO-split_07-08-17_Piano-e_1-06_wav--3.midi | 10 augmented versions | Avg: 2.2s/track | ETA: 0.5min
[7/19] MIDI-Unprocessed_058_PIANO058_MID--AUDIO-split_07-07-17_Piano-e_2-02_wav--4.midi | 10 augmented versions | Avg: 1.9s/track | ET

In [2]:
dataset = load_dataset("test_set/chunk_0.pkl")
print(f"Dataset loaded: {len(dataset)} tracks")
# Track Index: 981, Time: 46.505806393500684 returned less then 3 segment tokens!!

Dataset loaded: 1278 tracks


In [3]:
fig = plot_track_segments(track_idx=1, show=[100, 200])
fig.show()

Loading: maestro-v3.0.0\2008\MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.midi


In [3]:
model = Model.load('HPC_model.pt', device=device)

In [3]:
dict = evaluate(
    model=Model.load(path="alpha_0.pt", device=device),
    dataset=dataset,
    num_samples=100,
    device=device,
    threshold=0.3,
    seed=0
)
print(dict)

{'correlation': np.float64(-0.02245402646990418), 'binary': {'tp': np.int64(68036), 'tn': np.int64(1548), 'fp': np.int64(28692), 'fn': np.int64(1724)}, 'num_tokens_ratio': np.float64(2.523698076223493)}


In [6]:
model = Model(528, 6, 6, 8, 2112, 0.1).to(device)

In [ ]:
train(
    batch_size=64,
    lr=1e-4,
    num_steps=500,
    device=device,
    model=model,
    print_every=1,
    dataset=dataset,
    model_path="nocturnes_unnormalized.pt",
    alpha=0
)

Using provided model
Creating dataloader...
Starting training loop...
Step 1/500
  Total: 0.6375 | Segment: 0.0000 | Param: 0.6375
  Height: 0.0653 | Amount: 0.0124 | Time: 0.9889
Step 2/500
  Total: 0.9985 | Segment: 0.0000 | Param: 0.9985
  Height: 0.0972 | Amount: 0.0155 | Time: 1.5771
Step 3/500
  Total: 0.7791 | Segment: 0.0000 | Param: 0.7791
  Height: 0.0873 | Amount: 0.0161 | Time: 1.1768
Step 4/500
  Total: 0.7765 | Segment: 0.0000 | Param: 0.7765
  Height: 0.0724 | Amount: 0.0150 | Time: 1.2335
Step 5/500
  Total: 0.7534 | Segment: 0.0000 | Param: 0.7534
  Height: 0.0764 | Amount: 0.0137 | Time: 1.1738
Step 6/500
  Total: 0.7340 | Segment: 0.0000 | Param: 0.7340
  Height: 0.0707 | Amount: 0.0134 | Time: 1.1584
Step 7/500
  Total: 0.7313 | Segment: 0.0000 | Param: 0.7313
  Height: 0.0770 | Amount: 0.0118 | Time: 1.1312
Step 8/500
  Total: 0.6545 | Segment: 0.0000 | Param: 0.6545
  Height: 0.0719 | Amount: 0.0144 | Time: 0.9928
Step 9/500
  Total: 0.6410 | Segment: 0.0000 | Par

KeyboardInterrupt: 

In [ ]:
train_exhaustively(
    batch_size=50,
    lr=3e-4,
    num_steps=4000,
    device="cuda",
    model=Model(528, 6, 6, 8, 2112, 0.1).to(device),
    print_every=2000,
    model_path="complete_392000.pt",
    dataset_path="complete_dataset",
    accumulation_steps=2,
    num_rotations=2,
    num_workers=4,
    alpha=0.7,
    add_checkpoints=10000,
    record_loss=100
    )

# 14700 nach 190min

Found 49 chunk files in complete_dataset
Training: warmup=19600, effective_batch=50x2=100, workers=3, rotations=2
Total steps: 4000 steps/chunk × 49 chunks × 2 rotations = 392000 steps
Optimizations: mixed_precision=ON, cudnn_benchmark=ON
Loading chunk 1/49...
Creating dataloader for chunk 1...
Starting training on chunk 1...


KeyboardInterrupt: 

In [41]:
fig = visualize_teacher_forcing(model, dataset)
fig.show()

In [ ]:
fig = visualize_model(
    model=model,
    dataset=dataset,
    num_plots=12,
    device=device,
    exclude_context=False,
    show_notes=False,
    generate=True,
    ground_truth=True,
    seed=0
    )

In [47]:
fig.show()

In [10]:
import pickle
with open("saves/HPC_model.pt_loss_history.pkl", 'rb') as f:
    loss_history = pickle.load(f)


print(loss_history[0])
loss_history = [{'step': 0, 'losses': {'total': None, 'segment': None, 'param': None}, 'lr': 0}] + loss_history
save_pkl(loss_history, "HPC_loss.pkl")

fig = plot_loss("HPC_loss.pkl", detailed=True, plot_lr=True, title="Training Loss")
fig.show()
plot_loss("HPC_model.pkl", detailed=True, plot_lr=True, title="Training Loss", width=300, height=500)

{'step': 500, 'losses': {'total': 0.7327656792998314, 'segment': 0.2767288130819798, 'param': 1.6320882947444917}, 'lr': 2.5700500000000002e-05}


FileNotFoundError: [Errno 2] No such file or directory: 'saves\\HPC_model.pkl'

In [7]:
evaluate(
    model=model,
    dataset=dataset,
    num_samples=100,
    device=device,
    threshold=0.5,
    seed=None
    )

c:\Users\edgar\Documents\Studium\Mathe\Bachelor\Code\ML_with_Torch_venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\edgar\Documents\Studium\Mathe\Bachelor\Code\ML_with_Torch_venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\edgar\Documents\Studium\Mathe\Bachelor\Code\codebase\evaluate.py:22: RuntimeWarning: invalid value encountered in scalar divide
  recall = tp / (tp + fn)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 1000